In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import mean_squared_error

from src.constants import *

from src.utility.personal import *
from src.utility.structure import *
from src.utility.storage import *
from src.utility.math import *

from src.avgs import *
#from src.trajectory import *

from src.rolling_data import RollingData
from src.per_game_model import PerGameModel
from src.init_DFs.per_game import PerGameInit
from src.init_DFs.next_game import NextGameInit

color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

pgModel = PerGameModel()
initPG = PerGameInit()
initNextGame = NextGameInit()

In [13]:
season = "all"
situation = "5on5"
df = primary_csv(season, situation)

num_games = 10
add_objs=["season", "game_number"]
suffix=False

df

,game_id,team,opp_team,season,game_number,next_game_id,score,opp_score,win,reg_win,...,scoreAdjustedShotsAttempts,unblockedShotAttempts,scoreAdjustedUnblockedShotAttempts,dZoneGiveaways,xGoalsFromxReboundsOfShots,xGoalsFromActualReboundsOfShots,reboundxGoals,totalShotCredit,scoreAdjustedTotalShotCredit,scoreFlurryAdjustedTotalShotCredit
0,2014-10-08BOSPHI,BOS,PHI,2014,1,2014-10-09DETBOS,2,1,1,1,...,50.124,37,37.213,0,0.290,0.486,0.510,1.268,1.242,1.237
1,2014-10-08BOSPHI,PHI,BOS,2014,1,2014-10-09PHINJD,1,2,0,0,...,33.948,21,20.846,3,0.179,0.000,0.000,1.102,1.128,1.126
2,2014-10-08CGYVAN,CGY,VAN,2014,1,2014-10-09EDMCGY,2,4,0,0,...,35.342,28,26.068,5,0.252,0.215,0.253,1.316,1.226,1.220
3,2014-10-08CGYVAN,VAN,CGY,2014,1,2014-10-11VANEDM,4,2,1,1,...,44.930,31,33.784,4,0.271,0.043,0.043,1.411,1.538,1.513
4,2014-10-08LAKSJS,LAK,SJS,2014,1,2014-10-11ARILAK,0,4,0,0,...,49.719,43,38.718,8,0.367,0.370,0.370,1.608,1.452,1.438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21541,2023-04-13WSHNJD,WSH,NJD,2022,82,NaN,4,5,0,0,...,42.870,35,36.282,4,0.302,0.423,0.423,2.064,2.093,2.082
21542,2023-04-14CBJBUF,BUF,CBJ,2022,82,NaN,5,2,1,1,...,68.967,50,52.221,0,0.565,0.953,0.953,3.795,4.006,3.909
21543,2023-04-14CBJBUF,CBJ,BUF,2022,82,NaN,2,5,0,0,...,47.015,38,36.993,2,0.433,1.704,1.704,2.317,2.250,2.238
21544,2023-04-14NSHCOL,COL,NSH,2022,82,NaN,4,3,1,1,...,57.398,38,41.313,4,0.390,0.316,0.316,2.699,2.946,2.897


In [12]:
export_csv(pretty_df(df), season, situation, '5on5', False)

In [39]:
l = len(df)
df.iloc[2585:l]

,game_id,team,opp_team,season,next_game_id,score,opp_score,win,game_number,reg_win,...,scoreAdjustedShotsAttempts,unblockedShotAttempts,scoreAdjustedUnblockedShotAttempts,dZoneGiveaways,xGoalsFromxReboundsOfShots,xGoalsFromActualReboundsOfShots,reboundxGoals,totalShotCredit,scoreAdjustedTotalShotCredit,scoreFlurryAdjustedTotalShotCredit
2585,2023-04-13TBLDET,TBL,DET,2022,NaN,5,0,1,75,1,...,52.986,37,38.341,4,0.441,0.628,0.628,2.756,2.777,2.714
2586,2023-04-13WSHNJD,NJD,WSH,2022,NaN,5,4,1,75,0,...,67.234,51,49.959,3,0.556,0.955,0.955,3.182,3.196,3.139
2587,2023-04-13WSHNJD,WSH,NJD,2022,NaN,4,5,0,82,0,...,42.870,35,36.282,4,0.302,0.423,0.423,2.064,2.093,2.082
2588,2023-04-14CBJBUF,BUF,CBJ,2022,NaN,5,2,1,82,1,...,68.967,50,52.221,0,0.565,0.953,0.953,3.795,4.006,3.909
2589,2023-04-14CBJBUF,CBJ,BUF,2022,NaN,2,5,0,82,0,...,47.015,38,36.993,2,0.433,1.704,1.704,2.317,2.250,2.238
2590,2023-04-14NSHCOL,COL,NSH,2022,NaN,4,3,1,82,1,...,57.398,38,41.313,4,0.390,0.316,0.316,2.699,2.946,2.897
2591,2023-04-14NSHCOL,NSH,COL,2022,NaN,3,4,0,80,0,...,34.227,25,23.750,3,0.288,0.400,0.400,1.420,1.339,1.328


In [91]:
import numpy as np
import pandas as pd

def linear_coef(x, y, sample_size):
    if sample_size > len(x):
        return [np.nan, np.nan]

    mean_x = x.mean()
    mean_y = y.mean()

    # Calculate cross-deviation and deviation about x
    SS_xy = np.sum(y.copy() * x.copy()) - sample_size * mean_y * mean_x
    SS_xx = np.sum(x.copy() * x.copy()) - sample_size * mean_x * mean_x

    # Calculate regression coefficients
    b_1 = SS_xy / SS_xx
    b_0 = mean_y - b_1 * mean_x

    return [b0, b1]


def linear_coefs_grouped(x, y_group, sample_size):
    B = pd.DataFrame(index=["b0", "b1"], columns=y_group.columns)
    if sample_size > len(y_group):
        return B
    for col in B: 
        coefs = linear_coef(x, y_group[col].to_numpy(), sample_size)
        B[col] = coefs
    return B


In [46]:
# info: get linear regression of trajectory for previous X games
# includes the current game's data (:
def trajectory_linear(df, num_games, add_objs=["season", "game_number"], suffix=False):
    b0 = pd.DataFrame()
    b1 = pd.DataFrame()

    obj_cols = list(df.select_dtypes(include=["object"]).columns) + add_objs
    data_cols = [col for col in df.columns if col not in obj_cols]
    x = np.array(list(range(0, num_games)))

    for season in df["season"].unique():
        for team in df["team"].unique():
            group = df[((df["season"] == season) & (df["team"] == team))].copy()
            df_objs = group[obj_cols].copy()
            df_data = group[data_cols].copy()

            # Retrieve Linear Coefficients for each game
            for game in range(len(group)):
                start_index = max(0, game - num_games + 1)
                B = linear_coefs_grouped(x.copy(), df_data.loc[start_index:game+1].copy(), num_games)
                b0 = pd.concat(
                    [b0.copy(), pd.concat([df_objs.iloc[game].copy(), B.loc['b0'].copy()], axis=0)],
                    ignore_index=True,
                )
                b1 = pd.concat(
                    [b1.copy(), pd.concat([df_objs.iloc[game].copy(), B.loc['b1'].copy()], axis=0)],
                    ignore_index=True,
                )
    if suffix:
        for col in data_cols:
            b0 = rename_col(b0.copy(), col, f"{col}_{num_games}_b0")
            b1 = rename_col(b1.copy(), col, f"{col}_{num_games}_b1")
    return [b0, b1]
    #return [organize(b0), organize(b1)]


In [137]:
def linear_coef(x, y, sample_size):
    mean_x = x.mean()
    mean_y = y.mean()
    # Calculate cross-deviation and deviation about x
    SS_xy = np.sum(y.copy() * x.copy()) - sample_size * mean_y * mean_x
    SS_xx = np.sum(x.copy() * x.copy()) - sample_size * mean_x * mean_x
    # Calculate regression coefficients
    b1 = SS_xy / SS_xx
    b0 = mean_y - b1 * mean_x
    return [b0, b1]

def linear_coefs_grouped(x, y_group, sample_size):
    B = pd.DataFrame(index=["b0", "b1"], columns=y_group.columns)
    if sample_size != len(y_group):
        print(len(y_group))
        return B
    for col in B: 
        coefs = linear_coef(x, y_group[col].values, sample_size)
        B[col] = coefs
    return B

num_games = 4
df = df[(df["team"]=="CAR")].copy()
df = df[(df["game_number"] <= 20)]
b0 = pd.DataFrame()
b1 = pd.DataFrame()

obj_cols = list(df.select_dtypes(include=["object"]).columns) + add_objs
data_cols = [col for col in df.columns if col not in obj_cols]
x = np.array(list(range(0, num_games)))

for season in df["season"].unique():
    for team in df["team"].unique():
        group = organize(df[((df["season"] == season) & (df["team"] == team))].copy())
        df_objs = group[obj_cols].copy()
        df_data = group[data_cols].copy()
        for game in range(len(group)):
            start_index = max(0, game - num_games + 1)
            B = linear_coefs_grouped(x.copy(), df_data.loc[start_index:game].copy(), num_games)
            
            b0 = pd.concat(
                [b0.copy(), pd.concat([df_objs.iloc[game].copy(), B.loc['b0']], axis=0)],
                ignore_index=True,
            )
            print(df_objs.iloc[game].copy())
            #print(pd.concat([df_objs.iloc[game].copy(), B.loc['b0']]))

#B = linear_coefs_grouped(x, test_df.iloc[0:12].select_dtypes(exclude=["object"]), num_games)
#B = trajectory_linear(df, num_games)


1
game_id         2022-10-12CARCBJ
team                         CAR
opp_team                     CBJ
next_game_id    2022-10-14SJSCAR
game_date             2022-10-12
game_time               19:00:00
situation                   5on5
season                      2022
game_number                    1
Name: 0, dtype: object
2
game_id         2022-10-14SJSCAR
team                         CAR
opp_team                     SJS
next_game_id    2022-10-17SEACAR
game_date             2022-10-14
game_time               22:30:00
situation                   5on5
season                      2022
game_number                    2
Name: 1, dtype: object
3
game_id         2022-10-17SEACAR
team                         CAR
opp_team                     SEA
next_game_id    2022-10-20EDMCAR
game_date             2022-10-17
game_time               22:00:00
situation                   5on5
season                      2022
game_number                    3
Name: 2, dtype: object
game_id         2022-10-20EDMCAR
t

In [ ]:
# Retrieve Linear Coefficients for each game
        for game in range(len(group)):
            start_index = max(0, game - num_games + 1)
            B = linear_coefs_grouped(x.copy(), df_data.loc[start_index:game+1].copy(), num_games)
            b0 = pd.concat(
                [b0.copy(), pd.concat([df_objs.iloc[game].copy(), B.loc['b0'].copy()], axis=0)],
                ignore_index=True,
            )
            b1 = pd.concat(
                [b1.copy(), pd.concat([df_objs.iloc[game].copy(), B.loc['b1'].copy()], axis=0)],
                ignore_index=True,
            )
